In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup, UnicodeDammit
import pandas as pd
import numpy as np
import import_madison_health as mad

### Basic Selenium Interface:

In [80]:
path_to_chromedriver = '/Users/tracy/Desktop/chromedriver' # change path as needed
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [265]:
#navigate to main page
search_term='a'
url = 'https://elam.cityofmadison.com/HealthInspections/Default.aspx?AcceptsCookies=1'
browser.get(url)

search_content = browser.find_element_by_id('MainContent_txtSearchEstablishment')
search_content.clear()
search_content.send_keys(search_term)

# press search button
browser.find_element_by_id('MainContent_btnSearch').click()

## Design waterfall of search terms that will capture all Madison Restaurants:

In [187]:
# Download restaurant level info for all specified searches:
rest_list2 = []
search_list2 = ['a','e','i','o','u','y','b','c','d','f','g','h','j','k','l','m',
                'n','p','q','r','s','t','v','w','x','z',
                '608']

# The 'seen' list documents all restaurants that have already been captured 
# via prior search terms (and which should therefore be ignored/not counted)
seen2 = set()
for term in search_list2:
    restaurants = mad.search_restaurants(browser, term, seen2, False)
    rest_list2.append(restaurants)
    seen2.update(set(restaurants.keys()))
    print 'Term: %s    # New Restaurants: %d' % (term, len(restaurants))

Term: a    # New Restaurants: 1823
Term: e    # New Restaurants: 316
Term: i    # New Restaurants: 113
Term: o    # New Restaurants: 61
Term: u    # New Restaurants: 6
Term: y    # New Restaurants: 0
Term: b    # New Restaurants: 0
Term: c    # New Restaurants: 0
Term: d    # New Restaurants: 3
Term: f    # New Restaurants: 0
Term: g    # New Restaurants: 0
Term: h    # New Restaurants: 0
Term: j    # New Restaurants: 0
Term: k    # New Restaurants: 0
Term: l    # New Restaurants: 0
Term: m    # New Restaurants: 0
Term: n    # New Restaurants: 0
Term: p    # New Restaurants: 0
Term: q    # New Restaurants: 0
Term: r    # New Restaurants: 0
Term: s    # New Restaurants: 0
Term: t    # New Restaurants: 0
Term: v    # New Restaurants: 0
Term: w    # New Restaurants: 0
Term: x    # New Restaurants: 0
Term: z    # New Restaurants: 0
Term: 608    # New Restaurants: 1


#### As can be seen above, the vowel searches capture all but 4 Madison-area restaurants. 
"Jd's" and "608" are the only special cases that need to be added to the vowel search list

In [245]:
rest_list = []
search_list = ['a','e','i','o','u','y',"Jd's",'608']
seen = set()

# Grab restaurant-level information for all search terms in search_list
for term in search_list:
    restaurants = mad.search_restaurants(browser, term, seen, False)
    rest_list.append(restaurants)
    seen.update(set(restaurants.keys()))
    print 'Term: %s    # New Restaurants: %d' % (term, len(restaurants))
    
# Split 'a' search into 5 chunks (for parallelization)
# This is doen by making 5 'seen' files for the 'a' search, where 
# 4/5 of the id's are in each 'seen' file
r = rest_list[0].keys()
a_list = [r[i:i+365] for i in xrange(0,len(r),365)]
for i in xrange(len(a_list)):
    save_to_pickle([x for x in r if x not in a_list[i]], 'madison_a_%d.pkl' % i)
   
# Write out 'seen' files, which describe restaurant id's that have already been downloaded
# by previous searches. This step allows for parallelization of the final download process
r = rest_list[0]
save_to_pickle(r, '../yelp-health/data/mad/madison_a_full.pkl')
r.update(rest_list[1])
save_to_pickle(r.keys(), '../data/mad/madison_e_full.pkl')
r.update(rest_list[2])
save_to_pickle(r.keys(), '../data/mad/madison_i_full.pkl')
r.update(rest_list[3])
save_to_pickle(r.keys(), '../data/mad/madison_o_full.pkl')
r.update(rest_list[4])
save_to_pickle(r.keys(), '../data/mad/madison_u_full.pkl')
r.update(rest_list[6])
save_to_pickle(r.keys(), '../data/mad/madison_Jd_full.pkl')
r.update(rest_list[7])
save_to_pickle(r.keys(), '../data/mad/madison_608_full.pkl')

Term: a    # New Restaurants: 1823
Term: e    # New Restaurants: 316
Term: i    # New Restaurants: 113
Term: o    # New Restaurants: 61
Term: u    # New Restaurants: 6
Term: y    # New Restaurants: 0
Term: Jd's    # New Restaurants: 3
Term: 608    # New Restaurants: 1


## Exploratory Analysis

In [67]:
R_2 = mad.open_pickle('../data/mad/mad_health_e.pkl')

In [68]:
df_2 = pd.DataFrame.from_dict(R_2).T
df_2.head()

,address,inspections,name,type
MainContent_1015-674,"1328 S MIDVALE BLVD\nMADISON, WI 53711",{u'MainContent_2232643': {u'date': u'1/24/2012...,Letigre Lounge,Establisment Type not defined
MainContent_10150-65275,"5109 BARTON RD\nMADISON, WI 53711",{u'MainContent_2503686': {u'date': u'7/10/2014...,Ridgewood Pool,Primarily Restaurant
MainContent_10330-2461,"3555 E WASHINGTON AVE\nMADISON, WI 53704",{u'MainContent_2605482': {u'date': u'6/26/2015...,Pedro's,Primarily Restaurant
MainContent_1046-695,"1025 N SHERMAN AVE\nMADISON, WI 53704",{u'MainContent_2594426': {u'date': u'6/23/2015...,Esquire Club,Primarily Restaurant
MainContent_10788-2539,"1901 ABERG AVE\nMADISON, WI 53704",{u'MainContent_2578006': {u'date': u'4/16/2015...,Wiggies,Primarily Restaurant


In [72]:
# This restaurant was missing violations for the following inspections
keys = map(lambda x: set(x.keys()) if type(x)==dict else set(), df_2.inspections.tolist())
miss_list = ['MainContent_2610492']
miss_keys = []
for id_ in miss_list:
    miss_keys += df_2[[id_ in x for x in keys]].index.tolist()
miss_keys

['MainContent_LICFDM-2011-00201']

In [74]:
df_2.loc[['MainContent_LICFDM-2011-00201'],]

,address,inspections,name,type
MainContent_LICFDM-2011-00201,"515 S Midvale BLVD\nMadison, WI 53711",{u'MainContent_2318671': {u'date': u'10/22/201...,Luigi's Diner,Primarily Restaurant


In [81]:
# These 3 restaurants had the following error message when trying to access the link
# to their inspection-level information:
#     Exception: Message: stale element reference: 
#                element is not attached to the page document
problem_list = ["Luigi's Diner"]
df_replace = df_2[df_2.name.isin(problem_list)]
df_replace

,address,inspections,name,type
MainContent_LICFDM-2011-00201,"515 S Midvale BLVD\nMadison, WI 53711",{u'MainContent_2318671': {u'date': u'10/22/201...,Luigi's Diner,Primarily Restaurant


In [82]:
prob_dict = {}
#problem_list = ["Luigi"]

for term in problem_list:
    restaurants = mad.search_restaurants(browser, term)
    prob_dict.update(restaurants)
    print 'Term: %s    # New Restaurants: %d' % (term, len(restaurants))
    
df_prob = pd.DataFrame.from_dict(prob_dict).T
df_prob.head()


Scraping Restaurant Table: Luigi's Diner
Scraping Inspection Data
Number of new restaurants to scrape: 1
Term: Luigi's Diner    # New Restaurants: 1


,address,inspections,name,type
MainContent_LICFDM-2011-00201,"515 S Midvale BLVD\nMadison, WI 53711",{u'MainContent_2318671': {u'date': u'10/22/201...,Luigi's Diner,Primarily Restaurant


In [85]:
df_replace.inspections.values[0]['MainContent_2610492']

{'date': '8/3/2015',
 'result': 'No Reinspection Required',
 'type': 'Re-Inspection'}

In [86]:
df_prob[df_prob.index.isin(df_replace.index)].inspections.values[0]['MainContent_2610492']

{'date': '8/3/2015',
 'result': 'No Reinspection Required',
 'type': 'Re-Inspection',
 'violations': {'20a - COLD HOLDING - Not Corrected': ['Observation: Cheesecake 47F in display cooler.  Unit was in defrost cycle and staff had door open for extended time to clean interior and glass in unit.',
   'Corrective action: Maintain cold potentially hazardous foods at or below 41 degrees F, have service adjust defrost cycle to off hours.',
   'Code reference: WFC 3-501.16',
   'CDC Risk Factor: Improper Hold',
   'Action taken notes:  foods moved to other cooler, Unit completed defrost cycle,  and air temperature was 40F, unit appears to be functioning properly, defrost cycle timing+ cleaning lead to increased product temp. ',
   'Repeat Violation: No',
   'Corrected Onsite: Yes']}}

In [87]:
R_2_final = R_2.copy()
for id_ in df_replace.index:
    to_insert = df_prob.loc[id_,'inspections']
    R_2_final[id_]['inspections'] = to_insert

In [88]:
df_2_final = pd.DataFrame.from_dict(R_2_final).T

In [89]:
df_2_final[df_2_final.index.isin(df_replace.index)]

,address,inspections,name,type
MainContent_LICFDM-2011-00201,"515 S Midvale BLVD\nMadison, WI 53711",{u'MainContent_2318671': {u'date': u'10/22/201...,Luigi's Diner,Primarily Restaurant


In [90]:
mad.save_to_pickle(R_2_final, '../data/mad/mad_health_e_FINAL.pkl')

In [65]:
df_2_final.query("name == 'La Tolteca'").inspections.values[0]

{'MainContent_2239926': {'date': '2/28/2012',
  'result': 'Reinspection Required',
  'type': 'Routine Inspection',
  'violations': {'02a - EMPLOYEE HEALTH REPORT': ['Observation: Employees indicate they have been informed they must report symptoms, diagnosis of foodborne illness, or lesions containing pus to the person in charge, however, no signed forms available for inspector to verify.',
    'Corrective action: Person in charge shall provide training to food employees on required reporting of symptoms, diagnosis of foodborne illness and lesions containing pus.',
    'Code reference: WFC 2-201.11',
    'CDC Risk Factor: Other',
    'Score: 5',
    'Action taken notes:',
    'Repeat Violation: Yes',
    'Corrected Onsite: No'],
   '08f - HAND DRYING': ['Observation: No single-use toweling available at 2 out of 3 kitchen handsinks.',
    'Corrective action: Provide single-use toweling at handsink to facilitate proper handwashing.',
    'Code reference: WFC 6-301.12',
    'CDC Risk Fact

In [66]:
R_2_final['MainContent_73556-81811']

{'address': '317 S DIVISION ST\nSTOUGHTON, WI 53589',
 'inspections': {'MainContent_2322899': {'date': '11/13/2012',
   'result': 'No Reinspection Required',
   'type': 'Routine Inspection',
   'violations': {'46p - SANITIZER TEST KIT': ['Observation: A test kit for the sanitizer quaternary ammonia is not available or being used.',
     'Corrective action: Provide a test kit or other device for measuring the concentration of sanitizing solutions.',
     'Code reference: WFC 4-302.14',
     'Good Retail Practice',
     'Action taken notes:',
     'Repeat Violation: No',
     'Corrected Onsite: No'],
    '49p - PLUMBING GOOD REPAIR ': ['Observation: Repair the water leak in basement that is dripping from a waste pipe.',
     'Corrective action: Repair the plumbing system to conform to the State Uniform Plumbing Code.',
     'Code reference: WFC 5-205.15',
     'Good Retail Practice',
     'Action taken notes:',
     'Repeat Violation: No',
     'Corrected Onsite: No'],
    '53u - PHYSICA

# Merge all files:

In [3]:
file_list = ['0','1','2_FINAL','3_FINAL','4','608','e_FINAL','i','Jd','o','u']
R_list = []
for f in file_list:
    R_temp = mad.open_pickle('../data/mad/mad_health_%s.pkl' % f)
    R_list.append(pd.DataFrame.from_dict(R_temp).T)
    
R_full = pd.concat(R_list, axis=0)
R_full.info()
    

<class 'pandas.core.frame.DataFrame'>
Index: 2323 entries, MainContent_12775-2814 to MainContent_LICFDM-2012-00228
Data columns (total 4 columns):
address        2323 non-null object
inspections    2323 non-null object
name           2323 non-null object
type           2323 non-null object
dtypes: object(4)
memory usage: 90.7+ KB


In [4]:
R_full.head()

,address,inspections,name,type
MainContent_12775-2814,"1405 EMIL ST\nMADISON, WI 53713",{u'MainContent_2462414': {u'date': u'2/13/2014...,Licari's Bar & Grill,Primarily Restaurant
MainContent_1496-974,"119 W MAIN ST\nMADISON, WI 53703",{u'MainContent_2571346': {u'date': u'3/16/2015...,Paradise Lounge,Primarily Restaurant
MainContent_1497-981,"226 STATE ST\nMADISON, WI 53703",{u'MainContent_2231550': {u'date': u'1/18/2012...,Nick's Restaurant,Primarily Restaurant
MainContent_1660-1106,"425 N FRANCES ST\nMADISON, WI 53703",{u'MainContent_2478840': {u'date': u'4/29/2014...,Porta Bella,Primarily Restaurant
MainContent_19803-4575,"1029 S PARK ST\nMADISON, WI 53715",{u'MainContent_2325667': {u'date': u'11/29/201...,Oriental Shop,Retail Food Establishment


In [5]:
mad.save_to_pickle(R_full,'../data/mad/mad_health_FINAL.pkl')